In [1]:
import pandas as pd
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data/UnCleanedKaggle/financial_loan.csv')

In [3]:
df.columns

Index(['id', 'address_state', 'application_type', 'emp_length', 'emp_title',
       'grade', 'home_ownership', 'issue_date', 'last_credit_pull_date',
       'last_payment_date', 'loan_status', 'next_payment_date', 'member_id',
       'purpose', 'sub_grade', 'term', 'verification_status', 'annual_income',
       'dti', 'installment', 'int_rate', 'loan_amount', 'total_acc',
       'total_payment'],
      dtype='object')

In [4]:
df.isna().sum()

id                          0
address_state               0
application_type            0
emp_length                  0
emp_title                1438
grade                       0
home_ownership              0
issue_date                  0
last_credit_pull_date       0
last_payment_date           0
loan_status                 0
next_payment_date           0
member_id                   0
purpose                     0
sub_grade                   0
term                        0
verification_status         0
annual_income               0
dti                         0
installment                 0
int_rate                    0
loan_amount                 0
total_acc                   0
total_payment               0
dtype: int64

In [5]:
df.emp_title.fillna('No title',inplace=True)


In [6]:
df.nunique()

id                       38576
address_state               50
application_type             1
emp_length                  11
emp_title                28526
grade                        7
home_ownership               5
issue_date                  65
last_credit_pull_date      107
last_payment_date          102
loan_status                  3
next_payment_date          102
member_id                38576
purpose                     14
sub_grade                   35
term                         2
verification_status          3
annual_income             5096
dti                       2863
installment              15132
int_rate                   371
loan_amount                880
total_acc                   82
total_payment            19525
dtype: int64

In [7]:
df.address_state.value_counts()

address_state
CA    6894
NY    3701
FL    2773
TX    2664
NJ    1822
IL    1486
PA    1482
VA    1375
GA    1355
MA    1310
OH    1188
MD    1027
AZ     833
WA     805
CO     770
NC     759
CT     730
MI     685
MO     660
MN     592
NV     482
SC     464
WI     446
OR     436
AL     432
LA     426
KY     320
OK     293
KS     260
UT     252
AR     236
DC     214
RI     196
NM     183
HI     170
WV     167
NH     161
DE     110
WY      79
MT      79
AK      78
SD      63
VT      54
MS      19
TN      17
IN       9
ID       6
NE       5
IA       5
ME       3
Name: count, dtype: int64

In [8]:
df.application_type.value_counts() # kolumna do wyrzucenia

application_type
INDIVIDUAL    38576
Name: count, dtype: int64

In [9]:
THRESHOLD=10
low_representation = df.emp_title.value_counts()[df.emp_title.value_counts()<THRESHOLD].index
df.emp_title = df.emp_title.apply(lambda x: 'Other' if x in low_representation else x)

In [10]:
df.emp_title.value_counts()

emp_title
Other              33628
No title            1438
US Army              135
Bank of America      109
IBM                   67
                   ...  
Charles Schwab        10
Yale University       10
URS Corporation       10
Costco                10
Rite Aid              10
Name: count, Length: 169, dtype: int64

In [11]:
df['emp_length'].unique()

array(['< 1 year', '9 years', '4 years', '10+ years', '3 years',
       '5 years', '1 year', '6 years', '2 years', '7 years', '8 years'],
      dtype=object)

In [12]:
def parse(text: str) -> int:
        text = text.strip().lower()
        if text.startswith("<"):
            return 0                       
        elif "+" in text:
            return 10
        return int(text[0])

In [13]:
df['emp_years']=df['emp_length'].apply(lambda x: parse(x))
df['emp_years'].unique() 

array([ 0,  9,  4, 10,  3,  5,  1,  6,  2,  7,  8], dtype=int64)

In [14]:
df['grade'].unique() 

array(['C', 'E', 'B', 'A', 'D', 'F', 'G'], dtype=object)

In [15]:
def grades(grade: str)->int: 
    dictionary_grade=({
        'A':0, 
        'B':1, 
        'C':2, 
        'D':3,
        'E':4, 
        'F':5, 
        'G':6
    })
    if grade not in dictionary_grade.keys(): 
        raise ValueError("something wrong") 
    else: 
        return dictionary_grade[grade]

In [16]:
df['grade_numeric']=df['grade'].apply(lambda x: grades(x))
df['grade_numeric'].unique() 

array([2, 4, 1, 0, 3, 5, 6], dtype=int64)

In [17]:
df['sub_grade'].unique() 

array(['C4', 'E1', 'C5', 'B2', 'A1', 'C3', 'C2', 'A4', 'A5', 'B5', 'B4',
       'B3', 'B1', 'D1', 'A2', 'A3', 'D4', 'D2', 'C1', 'D3', 'E3', 'F1',
       'E2', 'E5', 'D5', 'E4', 'F2', 'G3', 'F3', 'G1', 'F4', 'G4', 'G2',
       'F5', 'G5'], dtype=object)

In [18]:
def sub_grades(sub_grade: str) -> float : 
    sub_str=sub_grade.strip()
    grade,sub=sub_str[0],int(sub_str[1])-1
    dictionary_grade=({
        'A':0 , 
        'B':1, 
        'C':2, 
        'D':3,
        'E':4, 
        'F':5, 
        'G':6
    })
    if grade not in dictionary_grade.keys(): 
        raise ValueError("something wrong") 
    else: 
        return int(dictionary_grade[grade]) + sub/5

In [19]:
df['sub_grade_numeric']=df['sub_grade'].apply(lambda x: sub_grades(x))
df['sub_grade_numeric'].unique() 

array([2.6, 4. , 2.8, 1.2, 0. , 2.4, 2.2, 0.6, 0.8, 1.8, 1.6, 1.4, 1. ,
       3. , 0.2, 0.4, 3.6, 3.2, 2. , 3.4, 4.4, 5. , 4.2, 4.8, 3.8, 4.6,
       5.2, 6.4, 5.4, 6. , 5.6, 6.6, 6.2, 5.8, 6.8])

In [20]:
df.home_ownership.value_counts()

home_ownership
RENT        18439
MORTGAGE    17198
OWN          2838
OTHER          98
NONE            3
Name: count, dtype: int64

In [21]:
df.loan_status.value_counts()

loan_status
Fully Paid     32145
Charged Off     5333
Current         1098
Name: count, dtype: int64

In [22]:
df.loc[df.home_ownership=='NONE','home_ownership']='OTHER'

In [23]:
df.purpose.value_counts()

purpose
Debt consolidation    18214
credit card            4998
other                  3824
home improvement       2876
major purchase         2110
small business         1776
car                    1497
wedding                 928
medical                 667
moving                  559
house                   366
vacation                352
educational             315
renewable_energy         94
Name: count, dtype: int64

In [24]:
df.columns

Index(['id', 'address_state', 'application_type', 'emp_length', 'emp_title',
       'grade', 'home_ownership', 'issue_date', 'last_credit_pull_date',
       'last_payment_date', 'loan_status', 'next_payment_date', 'member_id',
       'purpose', 'sub_grade', 'term', 'verification_status', 'annual_income',
       'dti', 'installment', 'int_rate', 'loan_amount', 'total_acc',
       'total_payment', 'emp_years', 'grade_numeric', 'sub_grade_numeric'],
      dtype='object')

In [25]:
df.verification_status.value_counts() # czy to czymś się różni? może i tak...

verification_status
Not Verified       16464
Verified           12335
Source Verified     9777
Name: count, dtype: int64

In [26]:
df.dti # DTI, czyli "Debt to Income" (dług do dochodu), to wskaźnik, który określa stosunek zobowiązań finansowych do wysokości dochodów.

0        0.0100
1        0.0535
2        0.2088
3        0.0540
4        0.0231
          ...  
38571    0.1986
38572    0.0458
38573    0.1734
38574    0.0009
38575    0.0600
Name: dti, Length: 38576, dtype: float64

In [27]:
def timeScraper(time_str: str) -> tuple[int, int, int, int]:
    dt = datetime.strptime(time_str, '%d-%m-%Y')  
    return dt.year, dt.month, dt.day, dt.weekday()


In [28]:
df[['issue_year', 'issue_month', 'issue_day', 'issue_weekday']] = df['issue_date'].apply(
    timeScraper).apply(pd.Series)

df[['last_credit_pull_year', 'last_credit_pull_month', 'last_credit_pull_day', 'last_credit_pull_weekday']] = df['last_credit_pull_date'].apply(
    timeScraper).apply(pd.Series)

df[['last_payment_year', 'last_payment_month', 'last_payment_day', 'last_payment_weekday']] = df['last_payment_date'].apply(
    timeScraper).apply(pd.Series)
df[['next_payment_year', 'next_payment_month', 'next_payment_day', 'next_payment_weekday']] = df['next_payment_date'].apply(
    timeScraper).apply(pd.Series)

In [29]:
df['term'].unique() 

array([' 60 months', ' 36 months'], dtype=object)

In [30]:
df['term_months']=df['term'].apply(lambda x : 60 if x==" 60 months" else 36) 
df['term_months'].unique()

array([60, 36], dtype=int64)

In [31]:
df.drop(columns=['id','emp_length','member_id','term', 'application_type', 'grade', 'sub_grade'],axis=1,inplace=True)

In [32]:
df.head()

,address_state,emp_title,home_ownership,issue_date,last_credit_pull_date,last_payment_date,loan_status,next_payment_date,purpose,verification_status,...,last_credit_pull_weekday,last_payment_year,last_payment_month,last_payment_day,last_payment_weekday,next_payment_year,next_payment_month,next_payment_day,next_payment_weekday,term_months
0,GA,Other,RENT,11-02-2021,13-09-2021,13-04-2021,Charged Off,13-05-2021,car,Source Verified,...,0,2021,4,13,1,2021,5,13,3,60
1,CA,Other,RENT,01-01-2021,14-12-2021,15-01-2021,Fully Paid,15-02-2021,car,Source Verified,...,1,2021,1,15,4,2021,2,15,0,36
2,CA,Other,RENT,05-01-2021,12-12-2021,09-01-2021,Charged Off,09-02-2021,car,Not Verified,...,6,2021,1,9,5,2021,2,9,1,36
3,TX,Other,MORTGAGE,25-02-2021,12-12-2021,12-03-2021,Fully Paid,12-04-2021,car,Source Verified,...,6,2021,3,12,4,2021,4,12,0,60
4,IL,Other,MORTGAGE,01-01-2021,14-12-2021,15-01-2021,Fully Paid,15-02-2021,car,Verified,...,1,2021,1,15,4,2021,2,15,0,36


In [33]:
df.to_csv("data/CleanedKaggle/financial_loan_cleaned.csv",index=False) 